In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/order_products__prior', './input/order_products__train', './input/orders',
                './input/products']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

# Instacart Recommender with `implicit`

In [1]:
# FIRST-AUTHOR: remove path printing
# import os

# def list_all_files_in(dirpath):
#     for dirname, _, filenames in os.walk(dirpath):
#         for filename in filenames:
#             print(os.path.join(dirname, filename))

# list_all_files_in('./input')

In [2]:
# Imports
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove plotting
# import implicit # implicit feedback recommendation library
# import matplotlib.pyplot as plt # visualization
# import seaborn as sns # wrapper to plt for ease of use
import time # timing
# FIRST-AUTHOR: remove ML code
# import scipy.sparse as sparse # sparse matrix support
# import pickle # Python object serialization

# from zipfile import ZipFile # ZIP file I/O
# from IPython.display import display # dataframe rendering, etc.
# from pathlib import Path

# color = sns.color_palette()
# sns.set_style('white')
# %matplotlib inline

# Load from ZIP files

In [3]:
# FIRST-AUTHOR: data already unzipped
# ds_dir = './input'

# with ZipFile(os.path.join(ds_dir,"aisles.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"departments.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"order_products__prior.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"order_products__train.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"orders.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"products.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()
# with ZipFile(os.path.join(ds_dir,"sample_submission.csv.zip"), 'r') as zipObj:
#     zipObj.extractall()

In [4]:
order_products_prior_df = pd.read_csv('./input/order_products__prior.scaled.csv')
order_products_train_df = pd.read_csv('./input/order_products__train.scaled.csv')
orders_df               = pd.read_csv('./input/orders.scaled.csv')
products_df             = pd.read_csv('./input/products.scaled.csv')

# Submission

In [5]:
# Merge orders and products
order_products_full_df = pd.concat([order_products_prior_df, order_products_train_df])
merged_order_products_df = pd.merge(order_products_full_df, products_df, on='product_id', how='left')

In [6]:
def get_user_products_df(path, orders_df, order_products_df):
    '''
    Generates a dataframe of users and their product purchases, and writes it to disk at the given path
    '''
    start = time.time()
    print('Creating user-product dataframe... ', end='')
    
    # Consider any "prior" orders and remove all columns except `user_id` from `df_orders`
    order_user_df = orders_df[['order_id', 'user_id']]
    
    # Remove all columns except order_id and user_id from orders_df and merge the above on `order_id` and remove `order_id`
    merged_df = pd.merge(order_products_df, order_user_df, on='order_id').drop('order_id',axis=1)
    reordered_user_products_df = merged_df.groupby(['user_id', 'product_id']).reordered.sum()
    user_products_df = pd.merge(merged_df, reordered_user_products_df, how='left', on=['user_id', 'product_id']).drop(['reordered_x', 'add_to_cart_order'], axis=1)
    
    # Write to disk
    user_products_df.to_csv(path, index_label=False)
    
    print(f'Completed in {round(time.time() - start, 2)}s')

# Build dataframe of users and their product purchases (Needed for building the utility matrix)

# FIRST-AUTHOR: make code always run
# REBUILD_MATRIX_DF_FULL = False
matrix_df_full_path = 'user_products_full.csv'
# FIRST-AUTHOR: make code always run
# if REBUILD_MATRIX_DF_FULL or not Path(matrix_df_full_path).is_file():
#     get_user_products_df(matrix_df_full_path, orders_df, order_products_full_df)
get_user_products_df(matrix_df_full_path, orders_df, order_products_full_df)

user_products_df = pd.read_csv(matrix_df_full_path)
user_products_df['user_id'] = user_products_df['user_id'].astype('category')
user_products_df['product_id'] = user_products_df['product_id'].astype('category')

Creating user-product dataframe... Completed in 66.92s


In [7]:
def build_product_user_matrix_full(path, user_products_df):
    '''
    Generates a utility matrix representing purchase history of users, and writes it to disk.
    Rows and columns represent products and users respectively.
    '''
# FIRST-AUTHOR: remove ML code
#     start = time.time()
#     print('Creating product-user matrix... ', end='')
    
#     product_user_matrix = sparse.coo_matrix((user_products_df['reordered_y'],
#                                             (user_products_df['product_id'].cat.codes.copy(),
#                                              user_products_df['user_id'].cat.codes.copy())))
#     sparse.save_npz(path, product_user_matrix)
    
#     print(f'Completed in {round(time.time() - start, 2)}s')
    _ = (user_products_df['reordered_y'],
                                            (user_products_df['product_id'].cat.codes.copy(),
                                             user_products_df['user_id'].cat.codes.copy()))

# FIRST-AUTHOR: make code always run
# REBUILD_MATRIX_FULL = False
matrix_full_path = 'product_user_matrix.npz'
# FIRST-AUTHOR: make code always run
# if REBUILD_MATRIX_FULL or not Path(matrix_full_path).is_file():
#     build_product_user_matrix_full(matrix_full_path, user_products_df)
build_product_user_matrix_full(matrix_full_path, user_products_df)

# FIRST-AUTHOR: remove ML code
# product_user_matrix_full = sparse.load_npz(matrix_full_path).tocsr()

In [8]:
# How sparse is the utility matrix?
# FIRST-AUTHOR: remove ML code
# def sparsity(matrix):
#     '''
#     Given a matrix, returns its sparsity
#     '''
#     total_size = matrix.shape[0] * matrix.shape[1]
#     actual_size = matrix.size
#     sparsity = (1 - (actual_size / total_size)) * 100
#     return sparsity

# sparsity(product_user_matrix_full)

In [9]:
# FIRST-AUTHOR: remove ML code
# def confidence_matrix(product_user_matrix, alpha):
#     '''
#     Given a utility matrix, returns the given matrix converted to a confidence matrix
#     (refer to http://yifanhu.net/PUB/cf.pdf for more details)
#     '''
#     return (product_user_matrix * alpha).astype('double')

# def build_lmf(product_user_matrix, **kwargs):
#     '''
#     Given the utility matrix and model parameters,
#     builds models and writes it to disk at a given path
#     '''
#     start = time.time()
    
#     # Build model
#     print(f'Building LMF model... ', end='')
#     model = implicit.lmf.LogisticMatrixFactorization()
#     model.approximate_similar_items = False
    
#     model.fit(product_user_matrix)
    
#     # Save model to disk
#     with open(kwargs['path'], 'wb+') as f:
#         pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
    
#     print(f'Completed in {round(time.time() - start, 2)}s')

# # Specify model params and build it
# lmf_params = {'random_state': 0}
# lmf_params['path'] = f'imf_benchmark_lmf.pkl'

# REBUILD_MODEL = True
# if REBUILD_MODEL or not Path(lmf_params['path']).exists():
#     build_lmf(product_user_matrix_full, **lmf_params)
# with open(lmf_params['path'], 'rb') as f:
#     lmf_model = pickle.load(f)

In [10]:
# Since the utility matrix is 0-indexed, the below dict is required to convert between `ids` and `indices`.
# For example, `product_id` 1 in the dataset is represented by the `0`th row of the utility matrix.

# Maps user_id: user index
u_dict = {uid : i for i, uid in enumerate(user_products_df['user_id'].cat.categories)}

# Maps product_index: product id
p_dict = dict(enumerate(user_products_df['product_id'].cat.categories))

In [11]:
orders_test_df = orders_df[orders_df.eval_set == 'test'][['user_id']]
relation_df = user_products_df[['user_id', 'product_id']]
relation_df.drop_duplicates(inplace=True)

/tmp/ipykernel_289128/2916545022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relation_df.drop_duplicates(inplace=True)


In [12]:
# FIRST-AUTHOR: remove ML code
# sparse_user_product_matrix = product_user_matrix_full.T.tocsr()
# N_REC = 100

In [13]:
def assign_recommendations(row):
    # print(f'Progress: {round((row.name + 1) * 100 / end, 2)}%...', end='\r', flush=True)
# FIRST-AUTHOR: remove ML code
#     return [(pid, score, rank) for rank, (pid, score) in enumerate(lmf_model.recommend(u_dict[row.user_id], sparse_user_product_matrix, N=N_REC), start=1)]
    return row.user_id

In [14]:
results_df = orders_test_df.reset_index(drop='index')
print('Recommending items... ', end='')
start = time.time()
results_df['products'] = results_df.apply(assign_recommendations, axis=1)
print(f'Completed in {round(time.time() - start, 2)}s')

Recommending items... Completed in 0.54s


In [16]:
results_df_new = results_df.explode('products').reset_index(drop='index').rename(columns={'products': 'product_id'})
# FIRST-AUTHOR: remove ML code
# results_df_new[['product_id', 'score', 'rank']] = pd.DataFrame(results_df_new['product_id'].tolist(), index=results_df_new.index)
results_df_new['product_id'] = results_df_new['product_id'].map(p_dict)
hasil = pd.merge(results_df_new, relation_df, how='inner', left_on=['user_id','product_id'], right_on=['user_id','product_id'])
hasil

,user_id,product_id
0,274,275.0
1,2090,2091.0
2,4064,4066.0
3,4864,4866.0
4,5874,5876.0
5,5910,5912.0
6,6084,6086.0
7,8498,8501.0
8,10746,10749.0
9,10811,10814.0


In [17]:
def clean_prediction(row):
    data = row.products
    data = str("".join(str(data))[1:-1].replace(',',' '))
    return data

In [18]:
r_hasil = hasil.groupby('user_id')['product_id'].apply(list).reset_index(name='products')
r_hasil['products'] = r_hasil.apply(clean_prediction, axis=1)
r_hasil

,user_id,products
0,274,275.0
1,2090,2091.0
2,4064,4066.0
3,4864,4866.0
4,5874,5876.0
5,5910,5912.0
6,6084,6086.0
7,8498,8501.0
8,10746,10749.0
9,10811,10814.0


In [19]:
submission_df = orders_df[orders_df.eval_set == 'test']
submission_df = submission_df[['order_id','user_id']]

sub_hasil = pd.merge(submission_df, r_hasil, how='outer', on='user_id').sort_values('user_id')
sub_hasil.fillna('None', inplace=True)
sub_hasil.drop('user_id', axis=1, inplace=True)
sub_hasil.to_csv('submission.csv', index=False)
sub_hasil

,order_id,products
0,2774568,None
1,329954,None
2,1528013,None
3,1376945,None
4,1356845,None
...,...,...
74995,2728930,None
74996,350108,None
74997,1043943,None
74998,2821651,None


In [20]:
sub_hasil[sub_hasil['products'] != 'None']

,order_id,products
97,2726815,275.0
754,2387149,2091.0
1435,1196675,4066.0
1746,1431569,4866.0
2143,1919482,5876.0
2159,1045861,5912.0
2226,431673,6086.0
3092,2094532,8501.0
3891,1957010,10749.0
3922,57286,10814.0
